In [1]:
import sys
import os
import math

import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../../data/processed/signalEUR_USD_Labelled_v1_processed.csv", index_col=0)

In [3]:
df["Date"] = pd.to_datetime(df["Date"])
# Labels 3 days ahead
df["label_3days"] = df["label"].shift(-3)
# Labels 5 days ahead
df["label_5days"] = df["label"].shift(-5)

In [4]:
df = df.dropna(axis = 0)
df = df.reset_index(drop=True)
df.head()

,Date,Price,Open,High,Low,Change %,diff_1,label,sma,ema,...,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci,label_3days,label_5days
0,2013-02-15,1.3363,1.3363,1.3394,1.3306,0.01%,0.0002,0.0,1.344735,1.336730,...,0.002701,0.005461,-0.002760,-0.000150,39.939646,1.364281,1.325189,-62.849696,1.0,0.0
1,2013-02-18,1.3352,1.3357,1.3377,1.3321,-0.08%,-0.0011,0.0,1.344925,1.335553,...,0.001906,0.004750,-0.002844,-0.004102,38.588551,1.363994,1.325856,-68.561252,1.0,1.0
2,2013-02-19,1.3388,1.3352,1.3397,1.3329,0.27%,0.0036,2.0,1.345260,1.338051,...,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,0.0,0.0
3,2013-02-20,1.3281,1.3387,1.3434,1.3270,-0.80%,-0.0107,1.0,1.345075,1.330396,...,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,2.0
4,2013-02-21,1.3189,1.3283,1.3291,1.3161,-0.69%,-0.0092,1.0,1.344140,1.321553,...,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,0.0,1.0


In [5]:
time = ["Date"]

columns = ["Price",  "sma", "ema", "cma", "macd",	"macd_s", 
           "macd_h", "roc", "rsi",	"Bollinger_up", "Bollinger_down", "cci"]

labels = ["label", "label_3days", "label_5days"]

features = df[time + columns + labels]

In [6]:
features.head()

,Date,Price,sma,ema,cma,macd,macd_s,macd_h,roc,rsi,Bollinger_up,Bollinger_down,cci,label,label_3days,label_5days
0,2013-02-15,1.3363,1.344735,1.336730,1.335315,0.002701,0.005461,-0.002760,-0.000150,39.939646,1.364281,1.325189,-62.849696,0.0,1.0,0.0
1,2013-02-18,1.3352,1.344925,1.335553,1.335311,0.001906,0.004750,-0.002844,-0.004102,38.588551,1.363994,1.325856,-68.561252,0.0,1.0,1.0
2,2013-02-19,1.3388,1.345260,1.338051,1.335408,0.001549,0.004109,-0.002561,-0.004906,45.544851,1.363601,1.326919,-57.039491,2.0,0.0,0.0
3,2013-02-20,1.3281,1.345075,1.330396,1.335211,0.000398,0.003367,-0.002969,-0.012785,32.802173,1.364052,1.326098,-90.131403,1.0,1.0,2.0
4,2013-02-21,1.3189,1.344140,1.321553,1.334782,-0.001242,0.002445,-0.003688,-0.012873,25.675429,1.366251,1.322029,-149.263447,1.0,0.0,1.0


In [56]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler

X = features.drop(labels= time + labels, axis=1)
Y = features[["label"]]

scaler = StandardScaler()
X = scaler.fit_transform(X)

onehot_encoder = OneHotEncoder()
Y = to_categorical(Y)

In [49]:
print(Y.shape)

(2311, 3)


In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=420)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(1848, 12) (463, 12)
(1848, 3) (463, 3)


In [58]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.losses import CategoricalCrossentropy

h_dim = 100
num_classes = 3

model = Sequential()
model.add(LSTM(units=h_dim, input_shape=(X_train.shape[1], 1)))
model.add(Dense(num_classes))
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 100)               40800     
                                                                 
 dense_14 (Dense)            (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________


In [61]:
epochs = 100

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Epoch 1/100
29/29 [==============================] - 0s 9ms/step - loss: 1.0059 - accuracy: 0.4838 - val_loss: 1.0473 - val_accuracy: 0.4428
Epoch 2/100
29/29 [==============================] - 0s 8ms/step - loss: 1.0084 - accuracy: 0.4713 - val_loss: 1.0464 - val_accuracy: 0.4708
Epoch 3/100
29/29 [==============================] - 0s 9ms/step - loss: 1.0030 - accuracy: 0.4843 - val_loss: 1.0556 - val_accuracy: 0.4579
Epoch 4/100
29/29 [==============================] - 0s 8ms/step - loss: 0.9984 - accuracy: 0.4908 - val_loss: 1.0419 - val_accuracy: 0.4557
Epoch 5/100
29/29 [==============================] - 0s 9ms/step - loss: 0.9971 - accuracy: 0.4848 - val_loss: 1.0413 - val_accuracy: 0.4492
Epoch 6/100
29/29 [==============================] - 0s 9ms/step - loss: 0.9937 - accuracy: 0.4919 - val_loss: 1.0403 - val_accuracy: 0.4557
Epoch 7/100
29/29 [==============================] - 0s 9ms/step - loss: 0.9909 - accuracy: 0.4973 - val_loss: 1.0376 - val_accuracy: 0.4536
Epoch 8/100
2

29/29 [==============================] - 0s 9ms/step - loss: 0.8092 - accuracy: 0.5979 - val_loss: 0.8553 - val_accuracy: 0.6004
Epoch 59/100
29/29 [==============================] - 0s 9ms/step - loss: 0.7954 - accuracy: 0.6126 - val_loss: 0.8446 - val_accuracy: 0.5853
Epoch 60/100
29/29 [==============================] - 0s 11ms/step - loss: 0.8087 - accuracy: 0.6104 - val_loss: 0.8320 - val_accuracy: 0.6026
Epoch 61/100
29/29 [==============================] - 0s 11ms/step - loss: 0.7900 - accuracy: 0.6228 - val_loss: 0.8454 - val_accuracy: 0.5875
Epoch 62/100
29/29 [==============================] - 0s 13ms/step - loss: 0.7814 - accuracy: 0.6190 - val_loss: 0.8228 - val_accuracy: 0.5940
Epoch 63/100
29/29 [==============================] - 0s 12ms/step - loss: 0.7820 - accuracy: 0.6180 - val_loss: 0.8335 - val_accuracy: 0.6091
Epoch 64/100
29/29 [==============================] - 0s 10ms/step - loss: 0.7901 - accuracy: 0.6266 - val_loss: 0.8457 - val_accuracy: 0.5767
Epoch 65/100
2